In [2]:
# Install Neo4j Python driver
!pip install neo4j pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 23.8 MB/s eta 0:00:00


In [3]:
from neo4j import GraphDatabase

# Replace these with your Aura credentials
NEO4J_URI = "neo4j+s://c49e21e2.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "5138749AZExP9pD8oVbe2dlqoYdH0Fnez1kJJxgR1K0"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def run_query(tx, query, params=None):
    return list(tx.run(query, params or {}))


In [ ]:
import pandas as pd

# Upload your CSV file to Colab, or load directly if hosted
from google.colab import files
uploaded = files.upload()

# Load dataset
df = pd.read_csv(next(iter(uploaded)))
df.fillna("", inplace=True)

# Simple Q&A parser (basic split)
def parse_dialogue(dialogue):
    qas = []
    lines = dialogue.split('\n')
    for line in lines:
        if "Doctor:" in line and "Patient:" in line:
            parts = line.split("Patient:")
            question = parts[0].replace("Doctor:", "").strip()
            answer = parts[1].strip()
            qas.append((question, answer))
        elif "Doctor:" in line:
            qas.append((line.replace("Doctor:", "").strip(), ""))
        elif "Patient:" in line and qas:
            qas[-1] = (qas[-1][0], line.replace("Patient:", "").strip())
    return qas


Saving labeled_data.csv to labeled_data.csv


In [ ]:
def build_knowledge_graph(tx, df):
    for idx, row in df.iterrows():
        case_id = f"Case_{row['ID']}"
        section = row['section_header']
        dialogue = row['dialogue']
        specialist = row['specialist_label']

        # Create Case node
        tx.run("MERGE (c:Case {id: $id, section: $section})", {"id": case_id, "section": section})

        # Extract symptoms from section_text (basic split for now)
        symptoms = [word.strip() for word in row['section_text'].split(',') if word]
        for symptom in symptoms:
            if symptom:
                tx.run("MERGE (s:Symptom {name: $name})", {"name": symptom})
                tx.run("MERGE (c:Case {id: $id}) "
                       "MERGE (s:Symptom {name: $name}) "
                       "MERGE (c)-[:HAS_SYMPTOM]->(s)", {"id": case_id, "name": symptom})

        # Parse Q&A pairs
        qas = parse_dialogue(dialogue)
        previous_question = None
        for i, (q, a) in enumerate(qas):
            qid = f"{case_id}_Q{i}"
            rid = f"{case_id}_A{i}"

            tx.run("MERGE (q:Question {id: $qid, text: $q})", {"qid": qid, "q": q})
            tx.run("MERGE (r:Response {id: $rid, text: $a})", {"rid": rid, "a": a})
            tx.run("MATCH (q:Question {id: $qid}), (r:Response {id: $rid}) "
                   "MERGE (q)-[:RESPONSE_TO]->(r)", {"qid": qid, "rid": rid})
            tx.run("MATCH (c:Case {id: $id}), (q:Question {id: $qid}) "
                   "MERGE (c)-[:CONTAINS_DIALOGUE]->(q)", {"id": case_id, "qid": qid})

            if previous_question:
                tx.run("MATCH (q1:Question {id: $q1}), (q2:Question {id: $q2}) "
                       "MERGE (q1)-[:FOLLOWED_BY]->(q2)", {"q1": previous_question, "q2": qid})
            previous_question = qid

        # Create Specialist node and relation
        tx.run("MERGE (s:Specialist {name: $name})", {"name": specialist})
        tx.run("MATCH (c:Case {id: $id}), (s:Specialist {name: $name}) "
               "MERGE (c)-[:REFERRED_TO]->(s)", {"id": case_id, "name": specialist})

with driver.session() as session:
    session.execute_write(build_knowledge_graph, df)


In [32]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

SIMILARITY_THRESHOLD = 0.6

# Step 1: Retrieve all cases with symptoms and dialogues
def fetch_cases(tx):
    query = """
    MATCH (c:Case)-[:HAS_SYMPTOM]->(s:Symptom)
    OPTIONAL MATCH (c)-[:CONTAINS_DIALOGUE]->(q:Question)-[:RESPONSE_TO]->(r:Response)
    WITH c, collect(DISTINCT s.name) as symptoms, collect({q: q.text, a: r.text}) as dialogue
    RETURN c.id as case_id, symptoms, dialogue
    """
    return list(tx.run(query))

In [33]:
# Step 2: Match user input with best case
def find_most_similar_case(user_symptoms, cases, accumulated_answers=None):
    user_vector = model.encode(user_symptoms, convert_to_tensor=True)
    max_score = -1
    best_case = None

    for case in cases:
        case_text = ", ".join(case['symptoms'])
        if accumulated_answers:
            case_text += " " + " ".join([f"{qa['q']} {qa['a']}" for qa in case['dialogue'][:len(accumulated_answers)]])
        case_vector = model.encode(case_text, convert_to_tensor=True)
        score = util.pytorch_cos_sim(user_vector, case_vector).item()
        if score > max_score:
            max_score = score
            best_case = case

    return best_case

In [34]:
# Step 3: Simulate dynamic consultation with branching
def simulate_consultation(cases, initial_case, user_symptoms):
    print("Starting consultation based on similar past case...\n")
    current_case = initial_case
    accumulated_answers = []

    for i, pair in enumerate(current_case['dialogue']):
        q, expected_a = pair['q'], pair['a']
        user_a = input(f"Doctor: {q}\nYou: ")
        accumulated_answers.append({'q': q, 'a': user_a})

        # Compute similarity between user answer and expected
        sim = util.pytorch_cos_sim(model.encode(user_a, convert_to_tensor=True),
                                   model.encode(expected_a, convert_to_tensor=True)).item()
        print(f"(Similarity to expected answer: {sim:.2f})")

        # Branching logic
        if sim < SIMILARITY_THRESHOLD:
            print("\n⚠️ Your response seems different from the expected pattern. Re-evaluating...\n")
            new_case = find_most_similar_case(user_symptoms, cases, accumulated_answers)
            if new_case and new_case['case_id'] != current_case['case_id']:
                print(f"🔄 Switching to a better matching case (Case ID: {new_case['case_id']})\n")
                current_case = new_case
                # Start again with updated case from current question
                remaining_dialogue = current_case['dialogue'][i:]
                for j, pair in enumerate(remaining_dialogue):
                    q, _ = pair['q'], pair['a']
                    user_a = input(f"Doctor: {q}\nYou: ")
                    accumulated_answers.append({'q': q, 'a': user_a})
                break  # End loop after switching
            else:
                print("No significantly better case found. Continuing...\n")

    # Final specialist recommendation
    with driver.session() as session:
        result = session.run("""
        MATCH (c:Case {id: $case_id})-[:REFERRED_TO]->(s:Specialist)
        RETURN s.name as specialist
        """, {"case_id": current_case['case_id']})
        specialist = result.single()['specialist']
        print(f"\n✅ Based on your responses, we recommend visiting a **{specialist}** specialist.")


In [35]:
# Main execution
with driver.session() as session:
    cases = session.read_transaction(fetch_cases)

user_input = input("📝 Please describe your symptoms: ")
initial_case = find_most_similar_case(user_input, cases)

if initial_case:
    simulate_consultation(cases, initial_case, user_input)
else:
    print("❌ Sorry, no similar past case found.")


<ipython-input-35-d707a8ff9465>:3: DeprecationWarning: read_transaction has been renamed to execute_read
  cases = session.read_transaction(fetch_cases)


📝 Please describe your symptoms: i have acne prone skin
Starting consultation based on similar past case...

Doctor: Okay, so your psych evaluation came out unremarkable. Do you have any other problems?
You: i have so many pimples
(Similarity to expected answer: 0.52)

⚠️ Your response seems different from the expected pattern. Re-evaluating...

🔄 Switching to a better matching case (Case ID: Case_813)

Doctor: Are you taking any medications?
You: no
Doctor: Okay, any other medicines?
You: no
Doctor: Okay.
You: okay

✅ Based on your responses, we recommend visiting a **Dermatologist** specialist.
